<a href="https://colab.research.google.com/github/jcannon04/colabs/blob/main/Byte_Workshop_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OpenAI and Functions Example

In [ ]:
!pip install openai

In [ ]:
import sqlite3
from openai import OpenAI
from google.colab import userdata
import json

database_path = "chinook.db"

def describe_database_schema():
    """
    Retrieves the database schema, including tables and their columns.

    Returns:
    - Dict: A dictionary with table names as keys and lists of column names as values.
    """
    schema = {}
    connection = sqlite3.connect(database_path)
    cursor = connection.cursor()

    # Retrieve all tables in the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    # Retrieve columns for each table
    for table in tables:
        table_name = table[0]
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        schema[table_name] = [column[1] for column in columns]  # column[1] is the name of the column

    connection.close()
    return json.dumps(schema, indent=4)

def execute_sql_query(query):
    """
    Executes an SQL query on the database and returns the results.

    Parameters:
    - query (str): The SQL query to execute.

    Returns:
    - List[Tuple]: The results of the query.
    """
    connection = sqlite3.connect(database_path)
    cursor = connection.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    connection.close()
    return json.dumps(results, indent=4)

def execute_function(name, args_dict):
    if name == "execute_sql_query":
        return execute_sql_query(**args_dict)
    elif name == "describe_database_schema":
        return describe_database_schema()
    else:
        raise ValueError(f"Function '{name}' not found")

tools = [
  {
      "type": "function",
      "function": {
          "name": "execute_sql_query",
          "description": "Executes an SQL query on the predefined database and returns the results. The database path is set within the application code.",
          "parameters": {
              "type": "object",
              "properties": {
                  "query": {
                      "type": "string",
                      "description": "The SQL query to execute."
                  }
              },
              "required": ["query"]
          }
      }
  },
  {
        "type": "function",
        "function": {
            "name": "describe_database_schema",
            "description": "Retrieves the database schema, including tables and their columns.",
            "parameters": {
                "type": "object",
                "properties": {}
            },
            "required": []
        }
    }
]


client = OpenAI(
  api_key=userdata.get("OPENAI_API_KEY")
)

messages=[
  {"role": "system", "content":

   """You are an SQL assistant built to answer database queries in natural language.
      Don't assume column or table names.
      Check the schema if unsure"""
  },
]

def start_chat(tools=None):

    while True:
        if messages and messages[-1]["role"] == 'function':
            print("Processing function call result...")
        else:
            prompt = input(">> ")
            if not prompt:
                break
            new_message = {"role": "user", "content": prompt}
            messages.append(new_message)

        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            tools=tools
        )

        assistant_message = response.choices[0].message

        if assistant_message.tool_calls:
          tool_calls = assistant_message.tool_calls
          for tool_call in tool_calls:
            tool = tool_call.function
            function_name = tool.name
            arguments = json.loads(tool.arguments)
            result = execute_function(function_name, arguments)
            messages.append({"role": "function", "tool_call_id": tool_call.id, "name": tool_call.function.name, "content": result})
        else:
            print(assistant_message.content)
            messages.append({"role": "user", "content": assistant_message.content})

start_chat(tools)